<a href="https://colab.research.google.com/github/caiocesarcosta/detection_transfer_learning_yolo_porject4/blob/main/detection_transfer_learning_yolo_porject4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import json
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import shutil
import yaml
import tensorflow as tf
from google.colab import drive

class YoloImageManagement:
    def __init__(self, num_images_to_download: int, category_names: list):
        self.__num_images_to_download = num_images_to_download
        self.category_names = category_names
        self.__data_dir = '/content/coco_yolo'
        self.__train_dir = os.path.join(self.__data_dir, 'images/train')
        self.__labels_dir = os.path.join(self.__data_dir, 'labels/train')
        self.img_width = 150 # Tamanho fixo da imagem, para manter a consistência
        self.__coco_data = None # Inicializa como None

        self.mountdriveGoogle()
        self.createDiretory()
        self.image_ids = []
        self.images_data = []
        self.__coco_data = self.downloadOrOpenAnnotations()  # carrega o arquivo de anotações

        if self.__coco_data:  # Executa o restante do código apenas se coco_data for carregado com sucesso
            self.image_to_categories = self.mapImageIDsToCategoryIDs() # Mapeia Ids das imagens para as categorias
            self.loadYoloDataset()

    def mountdriveGoogle(self):
        """Monta o Google Drive."""
        try:
            drive.mount('/content/drive', force_remount=True)
        except Exception as e:
            print(f"Erro ao montar o Google Drive: {e}")
            return None

    def createDiretory(self):
        """Cria os diretórios necessários."""
        os.makedirs(self.__train_dir, exist_ok=True)
        os.makedirs(self.__labels_dir, exist_ok=True)

    def convertBboxToYolo(self, bbox, img_width, img_height):
        """Converte bounding box do formato COCO para YOLO."""
        x, y, width, height = bbox
        x_center = ((x + (x + width)) / 2) / img_width
        y_center = ((y + (y + height)) / 2) / img_height
        width_norm = width / img_width
        height_norm = height / img_height
        return x_center, y_center, width_norm, height_norm

    def loadYoloDataset(self):
        """Função para baixar e salvar imagens com bounding boxes no formato YOLO."""

        # Itera sobre as imagens
        for image_data in self.images_data[:self.__num_images_to_download]: #limitar o número de imagens
            img_id = image_data['id']
            file_name = image_data['file_name']
            img_width = image_data['width']
            img_height = image_data['height']

            # Construir o arquivo de anotação (YOLO)
            annotation_file = os.path.join(self.__labels_dir, file_name.replace('.jpg', '.txt').replace('.png', '.txt'))

            # Baixar as imagens do site
            image_url = image_data['coco_url']
            try:
                response = requests.get(image_url)
                img = Image.open(BytesIO(response.content))
                img.save(os.path.join(self.__train_dir, file_name))

                annotation_content = []
                for annotation in self.__coco_data['annotations']:
                    if annotation['image_id'] == img_id and annotation['category_id'] in self.category_ids:
                        category_name = next((cat['name'] for cat in self.__coco_data['categories'] if cat['id'] == annotation['category_id']), None)
                        if category_name:
                            category_index = self.category_names.index(category_name)
                            bbox = annotation['bbox']
                            x_center, y_center, width_norm, height_norm = self.convertBboxToYolo(bbox, img_width, img_height)
                            annotation_content.append(f"{category_index} {x_center:.6f} {y_center:.6f} {width_norm:.6f} {height_norm:.6f}")

                # Verifica se gerou o conteudo
                if annotation_content:
                    print(f"Anotações geradas para {file_name}: {annotation_content}")
                    # Escreve no arquivo
                    with open(annotation_file, 'w') as f:
                        f.write('\n'.join(annotation_content))
                    print(f"Arquivo de anotação {annotation_file} criado.")
                else:
                    print(f"Nenhuma anotação encontrada para {file_name}")

            except Exception as e:
                print(f"Erro ao processar a imagem {file_name}: {e}")

        print("Imagens e anotações baixadas e convertidas com sucesso!")

    def downloadOrOpenAnnotations(self):
       """Baixar ou abre as informações da Annotation """
       annotations_url = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
       annotations_zip_path = os.path.join(self.__data_dir, 'annotations_trainval2017.zip')
       annotations_dir = os.path.join(self.__data_dir, 'annotations')

       if not os.path.exists(annotations_zip_path):
          print("Baixando anotações...")
          !wget -q "{annotations_url}" -O "{annotations_zip_path}"
          print("Extraindo anotações...")
          !unzip -q "{annotations_zip_path}" -d "{self.__data_dir}"

       instances_path = os.path.join(annotations_dir, 'instances_train2017.json')
       try:
            with open(instances_path, 'r') as f:
                coco_data = json.load(f)
                return coco_data
       except Exception as e:
            print(f"Erro ao abrir arquivo de anotação: {e}")
            return None

    def mapImageIDsToCategoryIDs(self):
        """Mapear IDs de imagens para IDs de categorias"""
        if self.__coco_data is None:
            print("Erro: coco_data não foi inicializado corretamente.")
            return

        image_to_categories = {}
        for ann in self.__coco_data['annotations']:
            image_id = ann['image_id']
            category_id = ann['category_id']
            if image_id not in image_to_categories:
                image_to_categories[image_id] = set() # Usando um set para ter elementos únicos
            image_to_categories[image_id].add(category_id)

        category_ids = [cat['id'] for cat in self.__coco_data['categories'] if cat['name'] in self.category_names]
        self.images_data = [img for img in self.__coco_data['images'] if img['id'] in image_to_categories and any(cat_id in image_to_categories[img['id']] for cat_id in self.category_ids)]

# ***************************************************************************************************
# Classe TransferLearningYolo
# ***************************************************************************************************
class TransferLearningYolo:
    def __init__(self, coco_file: YoloImageManagement):
        self.__repoYoloGitHub = 'https://github.com/ultralytics/yolov5'
        self.__pretrained_weights = 'yolov5s.pt' # Usa o modelo pequeno (mais rápido)
        self.img_width = coco_file.img_width
        self.__batch_size = 16
        self.__epochs = 10
        self.__cocoFile = coco_file
        self.coco_yaml = None

        self.cloneRepoistoryYoloGitHub()
        self.installDependency()
        self.createYAMLConfigurationFileToDataset()

    def cloneRepoistoryYoloGitHub(self):
        """Clone the yolo repository, from gitHub."""
        clone_command = f"git clone {self.__repoYoloGitHub}"
        os.system(clone_command)  # Use os.system para executar o comando

    def installDependency(self):
        """Install the dependecies."""
        os.chdir('yolov5')
        !pip install -r requirements.txt

    def createYAMLConfigurationFileToDataset(self):
        """Creates the yaml file to persist the information."""
        data = {
            'train': '/content/coco_yolo/images/train',
            'val': '/content/coco_yolo/images/train',  # Use os mesmos dados para validação (para simplificar)
            'nc': len(self.__cocoFile.category_names),  # Número de classes
            'names': self.__cocoFile.category_names  # Nomes das classes
        }

        with open('coco.yaml', 'w') as f:
            yaml.dump(data, f)

         # Imprimi o conteudo do arquivo.
        print("\nConteúdo do arquivo YAML:")
        !cat coco.yaml

    def trainModelYolo(self):
      train_command = f"python train.py --img {self.img_width} --batch {self.__batch_size} --epochs {self.__epochs} --data coco.yaml --weights {self.__pretrained_weights} --cache"
      os.system(train_command) # Use os.system

    def detectObjectImage(self):
        """Detect in the test image if the training was executed correctly. """
        test_image = '/content/yolov5/data/images/bus.jpg' #imagem de exemplo do yolo
        detect_command = f"python detect.py --weights runs/train/exp/weights/best.pt --img {self.img_width} --conf 0.5 --source \"{test_image}\""
        os.system(detect_command)  # Use os.system

    def train(self):
        """Orchestrates all the functions in the Train process."""
        self.trainModelYolo()
        self.detectObjectImage()

# inicialização dos objetos
num_images = 100  # quantidade de imagens
classes = ['person', 'dog', 'cat', 'car', 'bicycle', 'motorbike', 'traffic light', 'bird', 'truck']
yoloImages = YoloImageManagement(num_images, classes)
transferLearningYolo = TransferLearningYolo(yoloImages)
transferLearningYolo.train()

Mounted at /content/drive


AttributeError: 'YoloImageManagement' object has no attribute 'category_ids'